In [99]:
# SCRAPING GSOC ORGANISATIONS

In [100]:
import requests
import bs4

In [101]:
base_link = 'https://summerofcode.withgoogle.com'
organisations_links = []
org_links = []
org_tech = []
org_name = []
org_descp = []
org_topics = []
org_mail = []

In [102]:
res = requests.get('https://summerofcode.withgoogle.com/archive/2017/organizations/')

In [103]:
soup = bs4.BeautifulSoup(res.content,"lxml")

In [104]:
links = soup.find_all("a",{'class':"organization-card__link"})

In [ ]:
for link in links:
    organisations_links.append(base_link + link.get('href'))

In [ ]:
for org in organisations_links:
    res1 = requests.get(org)
    soup1 = bs4.BeautifulSoup(res1.content,"lxml")
    org_links.append(soup1.find("a",{'class':"org__link"}).get('href'))
    techn =  soup1.find_all("li" ,{'class':'organization__tag organization__tag--technology'})
    tl = []
    for tech in techn:
        tl . append(tech.string)
    org_tech.append(tech)
    meta_button = soup1.find_all("md-button",{"class":"md-primary org__meta-button"})
    org_mail.append(meta_button[-1].get('href')[7:])
    org_descp.append(decp.p.string)
    org_name.append( soup1.find('h3',{ 'class':"banner__title"}).string)


web services
ogc standards
java
javascript
web
python
c/c++
javascript
 shell script
static analysis
python
c/c++
vhdl
boinc
python
 javascript
machine learning
artificial intelligence
java
 android
golang
restful api
streaming
ruby on rails
scala
javascript/html/css
automated reasoning
android/ios
xcode
java
c
erlang
c++
xml
python
nlp
fpga
vhdl
c
linux
c++
c++
python
qt
c++
python
embedded
rtos
ros
javascript
babel
nodejs
java
linux
go
javascript
linux
beaglebone
python
c
erlang
elixir
lisp
xmpp
python
mobile
ios
android
ruby on rails
meteor.js
d3
elasticsearch
javascript 
opengl
opencl
python
c
c++
c++
c++11
c++14
boost
xia
 linux kernel
 c
 advanced data structures
c/c++
opencl
python
javascript
qt
python
django
javasc
javascipt
android
java
html5
ios
web
 javascript
java
web
reactjs
web services
c
python
raspberry pi
linux
visual studio
c++
python
object-storage
python
c/c++
machine learning
data analysis
parallel algorithms
chapel
python
high performance computing
c
c++
java
 jav

In [ ]:
#### creating database

In [ ]:
import sqlite3

conn = sqlite3.connect('GSOC_ORG.sqlite')
print("Opened database successfully");

conn.execute('''CREATE TABLE ORGS
         (
         organization   TEXT    NOT NULL,
         link            TEXT,
         description    TEXT,
         technologies   TEXT,
         contact    TEXT
         );''')
print("Table created successfully");
for i in range(len(organisations_links)):
    conn.execute("INSERT INTO ORGS VALUES('"+org_name[i]+ "','"+str(org_links[i])+"','"+str(org_descp[i])+"','"+str(org_tech[i])+"','"+str(org_mail[i])+"');")
print("successfully inserted")
conn.close()

In [1]:
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Mar/2018 21:38:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 21:38:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2018 21:39:57] "GET / HTTP/1.1" 200 -


In [ ]:
from flask import Flask, request
from flask_restful import Resource, Api
from sqlalchemy import create_engine
from json import dumps
from flask import jsonify

db_connect = create_engine('GSOC_ORG.db')
app = Flask(__name__)
api = Api(app)

class Employees(Resource):
    def get(self):
        conn = db_connect.connect() # connect to database
        query = conn.execute("select * from ORGS") # This line performs query and returns json result
        return {'employees': [i[0] for i in query.cursor.fetchall()], '',,,,} # Fetches first column that is Employee ID

# class Tracks(Resource):
#     def get(self):
#         conn = db_connect.connect()
#         query = conn.execute("select trackid, name, composer, unitprice from tracks;")
#         result = {'data': [dict(zip(tuple (query.keys()) ,i)) for i in query.cursor]}
#         return jsonify(result)

# class Employees_Name(Resource):
#     def get(self, employee_id):
#         conn = db_connect.connect()
#         query = conn.execute("select * from employees where EmployeeId =%d "  %int(employee_id))
#         result = {'data': [dict(zip(tuple (query.keys()) ,i)) for i in query.cursor]}
#         return jsonify(result)

api.add_resource(Organisations, '/organisations') # Route_1
# api.add_resource(Tracks, '/tracks') # Route_2
# api.add_resource(Employees_Name, '/employees/<employee_id>') # Route_3


if __name__ == '__main__':
     app.run(port='5002')
     

 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Mar/2018 22:11:41] "GET / HTTP/1.1" 404 -


In [1]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_marshmallow import Marshmallow
import os
import sys

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'GSOC_ORG.sqlite'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
ma = Marshmallow(app)


class org(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    organisation = db.Column(db.TEXT, unique=True)
    link = db.Column(db.TEXT)
    description = db.Column(db.TEXT)
    technologies = db.Column(db.TEXT)
    contact = db.Column(db.TEXT)
    def __init__(self, organisation, link, description, technologies, contact):
        self.organisation = organisation
        self.link = link
        self.description = description
        self.technologies = technologies
        self.contact = contact
        


class orgs(ma.Schema):
    class Meta:
        fields = ('organisation', 'link', 'description', 'technologies', 'contact')

orgs_schema = orgs(many=True)


@app.route("/organisations", methods=["GET"])
def get_orgs():
    all_orgs = org.query.all()
    result = orgs_schema.dump(all_orgs)
    return jsonify(result.data)


if __name__ == '__main__':
    app.run(debug=True)
    sys.exit(0)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
